# ModelIO

> 해당 자료 참고 링크 : [langchain v0.1 기준 Docs 페이지링크 : 현재 유지보수 지원하지 않는 버전](https://python.langchain.com/v0.1/docs/modules/)

- prompt
- chat models
- llms


## FewshotPromptTemplate


In [14]:

# 기본 방식
t = PromptTemplate(
    template="{country} 수도는?",
    input_variables = ["country"]
    )

t = PromptTemplate.from_template(
    "{country} 수도는?",
    )
t.format(country = "미국")


'미국 수도는?'

In [42]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate 
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate # 어떻게 대답해야하는지 LM에게 알려주기 위한 예제를 알려주는 것

model="gpt-4o-mini"
chat = ChatOpenAI(
    model_name=model,
    temperature=0.1,
    streaming = True, # LLM response가 생성되는대로 볼 수 있도록 함 /전체 응답이 끝나기 전에 한글자라도 나오면 볼수있음
    callbacks = [StreamingStdOutCallbackHandler(),] # 응답을 콘솔로 바로 print 해줌 / callback : 다양한 이벤트들을 감지할 수 있음(llm 시작 종료 등)
)
# 대화기록같은걸 DB에서 가져와서 예제로 넣어주면 형식화하여 답변을 더 잘할 것
# 보고서 형식/문법이라던가 대본 등등

# 예제가 없는 경우
# chat.predict("김치찌개 설명해줘")

examples = [{
    "question":"된장찌개 설명해줘",
    "answer" : """
    나라 : 한국
    주요 재료 : 된장, 두부, 호박
    비슷한 음식 : 청국장
    """
    },{
    "question":"김치찌개",
    "answer" : """
    나라 : 한국
    주요 재료 : 김치, 돼지고기, 양파 
    비슷한 음식 : 고추장찌개
    """
    },
    {
    "question":"훠거 설명해줘",
    "answer" : """
    나라 : 중국
    주요 재료 : 마라, 건두부, 목이버섯
    비슷한 음식 : 마라탕
    """
    },
]

# DB에서 가져올수도있고 직접 쓸수도있으므로 형식을 정해줘야함
example_tamplate = """
    Human : {question},
    AI : {answer}
"""
# example_prompt = PromptTemplate.from_template(example_tamplate) # 답변에 대한 템플릿을 지정해줄 수 있음
example_prompt = PromptTemplate.from_template("Human : {question}\nAI:{answer}") # 답변 형식을 지정해줌
prompt = FewShotPromptTemplate(
    example_prompt =example_prompt, 
    examples = examples,
    suffix  = "Human : {food}를 아니" ,# 뒤에 나올 고정말 
    input_variables = ['food'] # 유효성 검사 해줌 - prompt.format(food = "김치찌개") 에 food 가 없으면 에러남

)
# 랭체인이 알아서 예제 리스트들을 형식화할것

prompt.format(food = "파전")

chain = prompt | chat 

chain.invoke({"food":"파전"})


'Human : 된장찌개 설명해줘\nAI:\n    나라 : 한국\n    주요 재료 : 된장, 두부, 호박\n    비슷한 음식 : 청국장\n    \n\nHuman : 김치찌개\nAI:\n    나라 : 한국\n    주요 재료 : 김치, 돼지고기, 양파 \n    비슷한 음식 : 고추장찌개\n    \n\nHuman : 훠거 설명해줘\nAI:\n    나라 : 중국\n    주요 재료 : 마라, 건두부, 목이버섯\n    비슷한 음식 : 마라탕\n    \n\nHuman : 파전를 아니'

AI:
    나라 : 한국
    주요 재료 : 부침가루, 파, 해물 (주로 오징어, 새우 등)
    비슷한 음식 : 김치전

AIMessageChunk(content='AI:\n    나라 : 한국\n    주요 재료 : 부침가루, 파, 해물 (주로 오징어, 새우 등)\n    비슷한 음식 : 김치전')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate 
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate # 어떻게 대답해야하는지 LM에게 알려주기 위한 예제를 알려주는 것

model="gpt-4o-mini"
chat = ChatOpenAI(
    model_name=model,
    temperature=0.1,
    streaming = True, # LLM response가 생성되는대로 볼 수 있도록 함 /전체 응답이 끝나기 전에 한글자라도 나오면 볼수있음
    callbacks = [StreamingStdOutCallbackHandler(),] # 응답을 콘솔로 바로 print 해줌 / callback : 다양한 이벤트들을 감지할 수 있음(llm 시작 종료 등)
)
# 대화기록같은걸 DB에서 가져와서 예제로 넣어주면 형식화하여 답변을 더 잘할 것
# 보고서 형식/문법이라던가 대본 등등

# 예제가 없는 경우
# chat.predict("김치찌개 설명해줘")

examples = [{
    "question":"된장찌개 설명해줘",
    "answer" : """
    나라 : 한국
    주요 재료 : 된장, 두부, 호박
    비슷한 음식 : 청국장
    """
    },{
    "question":"김치찌개",
    "answer" : """
    나라 : 한국
    주요 재료 : 김치, 돼지고기, 양파 
    비슷한 음식 : 고추장찌개
    """
    },
    {
    "question":"훠거 설명해줘",
    "answer" : """
    나라 : 중국
    주요 재료 : 마라, 건두부, 목이버섯
    비슷한 음식 : 마라탕
    """
    },
]

# DB에서 가져올수도있고 직접 쓸수도있으므로 형식을 정해줘야함
example_tamplate = """
    Human : {question},
    AI : {answer}
"""
# example_prompt = PromptTemplate.from_template(example_tamplate) # 답변에 대한 템플릿을 지정해줄 수 있음
example_prompt = PromptTemplate.from_template("Human : {question}\nAI:{answer}") # 답변 형식을 지정해줌
prompt = FewShotPromptTemplate(
    example_prompt =example_prompt, 
    examples = examples,
    suffix  = "Human : {food}를 아니" ,# 뒤에 나올 고정말 
    input_variables = ['food'] # 유효성 검사 해줌 - prompt.format(food = "김치찌개") 에 food 가 없으면 에러남

)
# 랭체인이 알아서 예제 리스트들을 형식화할것

prompt.format(food = "파전")

chain = prompt | chat 

chain.invoke({"food":"파전"})
